In [1]:
import pandas as pd 
import xml.etree.ElementTree as et 
import os
import regex as re
import json
import os
import glob
from tqdm.autonotebook import tqdm

/Users/max/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [7]:
propfilelist = []
propdeffilelist = []
for root, dirs, files in os.walk("../../../Downloads/testcpi"):
    propfilelist = propfilelist + glob.glob(root+'/*.prop')
    propdeffilelist = propdeffilelist + glob.glob(root+'/*.propdef')

In [8]:
len(propfilelist) == len(propdeffilelist)

False

In [10]:
# propfilelist

In [11]:
columns = ['package', 'file', 'filename', 'key', 'value']
df = pd.DataFrame(columns=columns)
df.head()

,package,file,filename,key,value


In [12]:
properties = {}

In [13]:
reversedproperties = {}
for i in properties.keys():
    for j in properties.get(i):
        reversedproperties.update({j:i})

In [14]:
names = propfilelist
filenamemapp = {}
filenamemapp2 = {}
for i in names:
    for j in reversedproperties:
        if j in i:
            filenamemapp.update({i:reversedproperties.get(j)})
            filenamemapp2.update({i:j})
# df['package'] = df['filename'].map(filenamemapp)

In [15]:
def readFile(file):
    columns = ['package', 'file', 'filename', 'key', 'value']
    df = pd.DataFrame(columns=columns)
    
    filename = filenamemapp2.get(file)
    package = filenamemapp.get(file)
    with open(file, "r") as f:
        data = f.read()
        data = data[len("#Thu Sep 05 08:40:55 UTC 2019 "):]
        lines = data.split("\n")
        for l in lines:
            s = l.split("=")
            if len(s) >= 2:
                df = df.append({"package": package, "file": file, "filename": filename, \
                                "key": s[0], "value": s[1]},\
                                ignore_index=True)
    return df
    
def openPropfiles(filelist):
    dafrlist = []
    for i in filelist:
        dafrlist.append(readFile(i) )
    df = dafrlist[0]
    
    pbar = tqdm(total=len(dafrlist), leave=False)
    pbar.set_description(desc='Processing properties', refresh=True)
    pbar.update(1)
    
    for i in range(1, len(dafrlist)):
        df = df.append(dafrlist[i])
        pbar.update(1)
    pbar.close()
    
    df = df.reset_index()
    return df

In [16]:
df = openPropfiles(propfilelist)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 6 columns):
index       956 non-null int64
package     956 non-null object
file        956 non-null object
filename    956 non-null object
key         956 non-null object
value       956 non-null object
dtypes: int64(1), object(5)
memory usage: 44.9+ KB


In [18]:
def readXmlPropFile(file):
    xtree = et.parse(file)
    xroot = xtree.getroot()
    return iterxml(xroot, file)
    
    
def iterxml(xml, file):    
    filename = filenamemapp2.get(file)
    package = filenamemapp.get(file)
    df = pd.DataFrame(columns=columns)
    countproperty = 0
    for elem in list(xml):
        path = ""
        firsttabs = ""
        rootelem = ""
        if "}" in elem.tag:
            rootelem = elem.tag.split("}", 1)[1]
        else:
            rootelem = elem.tag

#         print(firsttabs + rootelem)
#         print(firsttabs + "  " + str(elem.attrib))
        rootid = elem.attrib.get("id")
        rootname = elem.attrib.get("name")
        path += "/"+rootelem
        info = ""
        infodict = {}
        if rootelem == "parameter":
            for prop in elem:
                
                if prop.tag == "additionalMetadata":
                    if len(list(prop)) > 1:
                        for adddata in prop:
                            text += '{ "' +adddata.tag + '":"' + adddata.text + '"}'
                    else:
                        text = "NoData"
                else:
                    text = str(prop.text)
                if text == "":
                    text = None
                if not prop.tag == "key":
                    info += ", "+ '"'+prop.tag+ '"' + ':' + '"'+ text+ '"'
                else:
                    info +=  '"' + prop.tag+ '"' + ':'+ '"' + text+ '"'
#             print(file)
#             print(info)
            infodict = json.loads("{" + info + "}")
        elif rootelem == "param_references":
            for prop in elem:
                infodict = prop.attrib
        infodict.update({"package":package,"file":file,"filename":filename})
#         print(infodict)
        df = df.append(infodict,ignore_index=True)
#       df = df.append({'package':None, 'file':None, 'filename':None, 'key':None, 'name':None, 'type':None, 'isRequired':None, \
#                'constraint':None, 'description':None, 'additionalMetadata':None, 'attribute_category':None,\
#                'attribute_id':None, 'attribute_uilabel':None, 'param_key':None},\
#                                 ignore_index=True)
    return df
    
def openPropDeffiles(filelist):
    dafrlist = []
    for i in filelist:
        dafrlist.append(readXmlPropFile(i) )
    df = dafrlist[0]
    
    pbar = tqdm(total=len(dafrlist), leave=False)
    pbar.set_description(desc='Processing properties', refresh=True)
    pbar.update(1)
    
    for i in range(1, len(dafrlist)):
        df = df.append(dafrlist[i])
        pbar.update(1)
    pbar.close()
    
#     df = df.reset_index()
    return df

In [19]:
df = openPropDeffiles(propdeffilelist)

/Users/max/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1079 entries, 0 to 0
Data columns (total 15 columns):
additionalMetadata    192 non-null object
attribute_category    84 non-null object
attribute_id          84 non-null object
attribute_uilabel     84 non-null object
constraint            935 non-null object
description           935 non-null object
file                  1079 non-null object
filename              0 non-null object
isRequired            935 non-null object
key                   935 non-null object
name                  935 non-null object
package               0 non-null object
param_key             84 non-null object
type                  935 non-null object
value                 0 non-null object
dtypes: object(15)
memory usage: 134.9+ KB


In [1]:
df.head(20)